<a href="https://colab.research.google.com/github/Mona1811k/Anomaly-Detection-in-CCTV-Footage-using-Deep-Learning-and-with-Alerting-Sytsem/blob/main/smart_ser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output,Video
import keras
import numpy as np
import cv2

In [ ]:
scvd_train_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train'
scvd_test_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test'
scvd_classes = ['Normal', 'Violence','Weaponized']

In [ ]:
# Define the base directory and categories
base_dir = '/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train'
categories = ['Normal', 'Violence', 'Weaponized']

In [ ]:
import os

# Function to count videos in each category
def count_videos(base_dir, categories):
    video_count = {}

    for category in categories:
        category_path = os.path.join(base_dir, category)

        if not os.path.exists(category_path):
            print(f"Directory for category '{category}' not found.")
            video_count[category] = 0
            continue

        # List all video files in the category directory
        video_files = [f for f in os.listdir(category_path) if f.endswith(('.mp4', '.avi', '.mkv'))]
        video_count[category] = len(video_files)

    return video_count

# Call the function and print the counts
video_counts = count_videos(base_dir, categories)

print("Video counts per category:")
for category, count in video_counts.items():
    print(f"{category}: {count} videos")


Video counts per category:
Normal: 872 videos
Violence: 970 videos
Weaponized: 832 videos


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 50

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
# Define the label mapping
label_mapping = {"Normal": 0, "Violence": 1, "Weaponized": 2}


In [ ]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
